# Arbitrary floating-point types

Since DFTK is completely generic in the floating-point type
in its routines, there is no reason to perform the computation
using double-precision arithmetic (i.e.`Float64`).
Other floating-point types such as `Float32` (single precision)
are readily supported as well.
On top of that we already reported[^HLC2020] calculations
in DFTK using elevated precision
from [DoubleFloats.jl](https://github.com/JuliaMath/DoubleFloats.jl)
or interval arithmetic
using [IntervalArithmetic.jl](https://github.com/JuliaIntervals/IntervalArithmetic.jl).
In this example, however, we will concentrate on single-precision
computations with `Float32`.

The setup of such a reduced-precision calculation is basically identical
to the regular case, since Julia automatically compiles all routines
of DFTK at the precision, which is used for the lattice vectors.
Apart from setting up the model with an explicit cast of the lattice
vectors to `Float32`, there is thus no change in user code required:

[^HLC2020]:
    M. F. Herbst, A. Levitt, E. Cancès.
    *A posteriori error estimation for the non-self-consistent Kohn-Sham equations*
    [ArXiv 2004.13549](https://arxiv.org/abs/2004.13549)

In [1]:
using DFTK

# Setup silicon lattice
a = 10.263141334305942  # lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Cast to Float32, setup model and basis
model = model_DFT(Array{Float32}(lattice), atoms, positions, [:lda_x, :lda_c_vwn])
basis = PlaneWaveBasis(model, Ecut=7, kgrid=[4, 4, 4])

# Run the SCF
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.900868892670                   -0.70    4.1
  2   -7.905413627625       -2.34       -1.52    1.0
  3   -7.905616760254       -3.69       -2.52    3.0
  4   -7.905618667603       -5.72       -3.34    2.6


To check the calculation has really run in Float32,
we check the energies and density are expressed in this floating-point type:

In [2]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             3.1022017 
    AtomicLocal         -2.1990583
    AtomicNonlocal      1.7296659 
    Ewald               -8.3978930
    PspCorrection       -0.2946220
    Hartree             0.5531532 
    Xc                  -2.3990662

    total               -7.905618667603

In [3]:
eltype(scfres.energies.total)

Float32

In [4]:
eltype(scfres.ρ)

Float32

!!! note "Generic linear algebra routines"
    For more unusual floating-point types (like IntervalArithmetic or DoubleFloats),
    which are not directly supported in the standard `LinearAlgebra` library of Julia
    one additional step is required: One needs to explicitly enable the generic versions
    of standard linear-algebra operations like `cholesky` or `qr`, which are needed
    inside DFTK by loading the `GenericLinearAlgebra` package in the user script
    (i.e. just add ad `using GenericLinearAlgebra` next to your `using DFTK` call).